In [1]:
import pandas as pd

# Load the CSV file
file_path = "AWT vs. NPS v4.csv"  # Update with your file path
data = pd.read_csv(file_path)

# Convert WAIT_TIME from seconds to minutes
data['WAIT_TIME_MINUTES'] = data['WAIT_TIME'] / 60

# Combine CREATED_DATE and CREATED_TIME into a single datetime column
data['CREATED_AT'] = pd.to_datetime(data['CREATED_DATE'] + ' ' + data['CREATED_TIME'])

# Extract DATE and HOUR for grouping
data['DATE'] = data['CREATED_AT'].dt.date
data['HOUR'] = data['CREATED_AT'].dt.hour

# Define a function to calculate the NPS score
def calculate_nps(group):
    total_responses = len(group)
    if total_responses == 0:  # Avoid division by zero
        return 0
    promoters = len(group[group['NET_PROMOTER_SCORE'] >= 9])  # Promoters: NPS of 9-10
    detractors = len(group[group['NET_PROMOTER_SCORE'] <= 6])  # Detractors: NPS of 0-6
    nps = ((promoters - detractors) / total_responses) * 100
    return nps

# Group by DATE and HOUR
grouped = data.groupby(['DATE', 'HOUR'])

# Calculate average wait time and NPS for each hour of each day
results = grouped.agg(
    AVG_WAIT_TIME_MINUTES=('WAIT_TIME_MINUTES', 'mean'),
    NPS_SCORE=('NET_PROMOTER_SCORE', calculate_nps)
).reset_index()

# Save the results to a CSV file
output_path = "Hourly_AWT_NPS.csv"  # Update with your desired output path
results.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")


KeyError: 'NET_PROMOTER_SCORE'